In [ ]:
# http://www.capo.gov.cy/capo/capo.nsf/All/B3C8BE19E2B8C3A8C2257FC6003EB33A/$file/BENEFICIARIES_2014_FINAL_v3.xlsx
# http://www.capo.gov.cy/capo/CAPO.nsf/All/D81AAD936F05B416C2257FC6003F385F/$file/%CE%A0%CE%BB%CE%B7%CF%81%CF%89%CE%BC%CE%AD%CF%82%20%CE%B4%CE%B9%CE%BA%CE%B1%CE%B9%CE%BF%CF%8D%CF%87%CF%89%CE%BD%202015%20-%2031-5-16%20.xlsx

In [4]:
!wget -nc 'http://www.capo.gov.cy/capo/capo.nsf/All/BD7C94C6B96F36D8C2257E5A00385E12/$$file/BENEFICIARIES_2014_FINAL_v3.xlsx'

--2016-05-06 11:07:37--  http://www.capo.gov.cy/capo/capo.nsf/All/BD7C94C6B96F36D8C2257E5A00385E12/$file/BENEFICIARIES_2014_FINAL_v3.xlsx
Resolving www.capo.gov.cy... 212.31.118.26
Connecting to www.capo.gov.cy|212.31.118.26|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3869432 (3.7M) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: 'BENEFICIARIES_2014_FINAL_v3.xlsx'

BENEFICIARIES_2014_ 100%[===================>]   3.69M   549KB/s    in 7.3s    

2016-05-06 11:07:44 (519 KB/s) - 'BENEFICIARIES_2014_FINAL_v3.xlsx' saved [3869432/3869432]



In [5]:
!curl -o cy_2015_raw.xlsx 'http://www.capo.gov.cy/capo/CAPO.nsf/All/D81AAD936F05B416C2257FC6003F385F/$file/%CE%A0%CE%BB%CE%B7%CF%81%CF%89%CE%BC%CE%AD%CF%82%20%CE%B4%CE%B9%CE%BA%CE%B1%CE%B9%CE%BF%CF%8D%CF%87%CF%89%CE%BD%202015%20-%2031-5-16%20.xlsx' -H 'Pragma: no-cache' -H 'DNT: 1' -H 'Accept-Encoding: gzip, deflate, sdch' -H 'Accept-Language: en-US,en;q=0.8,de;q=0.6' -H 'Upgrade-Insecure-Requests: 1' -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' -H 'Cache-Control: no-cache' -H 'Referer: http://www.capo.gov.cy/capo/CAPO.nsf/All/D81AAD936F05B416C2257FC6003F385F?OpenDocument' -H 'Connection: keep-alive' --compressed

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4455k  100 4455k    0     0   418k      0  0:00:10  0:00:10 --:--:--  439k:00:01  0:00:37  115k


In [ ]:
!wget -nc -O cy_2016.xlsx 'http://www.capo.gov.cy/capo/capo.nsf/05082a1738ff5899c225776f002ea4a8/AD063B69CE4D0D06C2258131002AA09F/$file/FINAL%20%CE%A0%CE%BB%CE%B7%CF%81%CF%89%CE%BC%CE%AD%CF%82%20%CE%94%CE%B9%CE%BA%CE%B1%CE%B9%CE%BF%CF%8D%CF%87%CF%89%CE%BD%202016-%2031.5.17.xlsx'

In [4]:
import pandas as pd

In [7]:
from slugify import slugify

def add_to_df(df):
    df['country'] = 'CY'
    df['currency'] = 'EUR'

    df['recipient_id'] = df.apply(lambda x: 'CY-%s' % x['CODE APPLICANT'] if pd.notnull(x['CODE APPLICANT']) else
                                            'CY-%s-%s' % (x['recipient_postcode'], slugify(
                                                '' if pd.isnull(x['recipient_name']) else x['recipient_name'])), 1)

    df['amount'] = df.apply(lambda x: x[['EAGF', 'EAFRD']].sum(), 1)
    df = df.drop(['CODE APPLICANT', 'EAFRD', 'EAGF'], 1)
    return df


def fix_df(df, year):
    original_columns = list(df.columns)
    translations = 'NAME OF BENEFICIARY	Subsidy by EAFRD	Subsidy by EAGF	Measure code	Description of Measure	POSTAL_CODE	TOWN'.split('\t')
    df = df.rename(columns=dict(zip(original_columns, translations)))
    df['year'] = year
    df = df.rename(columns={
        'NAME OF BENEFICIARY': 'recipient_name',
        'Subsidy by EAFRD': 'EAFRD',
        'Subsidy by EAGF': 'EAGF',
        'Description of Measure': 'scheme',
        'POSTAL_CODE': 'recipient_postcode',
        'TOWN': 'recipient_location',
    }).drop(['Measure code'], 1)
    return df


In [ ]:
df_2016 = pd.read_excel('cy_2016_raw.xlsx')
df_2016 = fix_df(df_2016, 2016)

In [12]:
df_2016.head()

,recipient_name,EAFRD,EAGF,scheme,recipient_postcode,recipient_location,ΠΕΡΙΓΡΑΦΗ ΜΕΤΡΟΥ/DESCRIPTION OF MEASURES,ΤΑΧΥΔΡΟΜΙΚΟΣ ΚΩΔΙΚΑΣ/POSTAL CODE,ΔΗΜΟΣ/ΕΠΑΡΧΙΑ/ΚΟΙΝΟΤΗΤΑ /MUNICIPALITY/DISTRICT/COMMUNITY,year,country,currency
0,ΑΝΑΣΤΑΣΗΣ ΧΡΙΣΤΟΔΟΥΛΟΥ,NaN,2547.15,2547.15,50215062320019,III.7,"Specific aid for bee-keeping - R.1308/13, Art....",8520,ΜΑΡΑΘΟΥΝΤΑ,2016,CY,EUR
1,ΟΜΑΔ. ΚΑΛΛΙΕΡΓΕΙΑ ΣΠΕ ΑΚΑΝΘΟΥΣ,NaN,NaN,65390.20,50405012141001,V/B.2.4,RD-Agrienvironmental measures,7575,ΛΑΡΝΑΚΑ,2016,CY,EUR
2,ΟΜΑΔ. ΚΑΛΛΙΕΡΓΕΙΑ ΣΠΕ ΑΚΑΝΘΟΥΣ,NaN,NaN,65390.20,50405012141001,V/B.2.4,RD-Agrienvironmental measures,7575,ΛΑΡΝΑΚΑ,2016,CY,EUR
3,ΣΕΔΙΓΕΠ ΠΕΡΙΦΕΡΕΙΑΣ ΠΟΛΗΣ ΧΡΥΣΟΧΟΥΣ ΛΤΔ,NaN,58343.69,58343.69,50208030000054,III.5,"Operational programmes - balance - R.1308/13, ...",8873,ΠΑΦΟΣ,2016,CY,EUR
4,ΣΕΔΙΓΕΠ ΠΕΡΙΦΕΡΕΙΑΣ ΠΟΛΗΣ ΧΡΥΣΟΧΟΥΣ ΛΤΔ,NaN,321118.93,321118.93,50208030000070,III.5,Exceptional support measures - producers organ...,8873,ΠΑΦΟΣ,2016,CY,EUR


In [11]:
df_2016 = add_to_df(df_2016)
df_2016.head()

KeyError: ('CODE APPLICANT', 'occurred at index 0')

In [40]:
df_2015 = pd.read_excel('cy_2015_raw.xlsx', sheetname='Πληρωμές δικαιούχων 2015')
original_columns = list(df_2015.columns)
translations = 'NAME OF BENEFICIARY	Subsidy by EAFRD	Subsidy by EAGF	Measure code	Description of Measure	POSTAL_CODE	TOWN'.split('\t')
df_2015 = df_2015.rename(columns=dict(zip(original_columns, translations)))
df_2015['year'] = 2015
df_2015 = df_2015.rename(columns={
    'NAME OF BENEFICIARY': 'recipient_name',
    'Subsidy by EAFRD': 'EAFRD',
    'Subsidy by EAGF': 'EAGF',
    'Description of Measure': 'scheme',
    'POSTAL_CODE': 'recipient_postcode',
    'TOWN': 'recipient_location',
}).drop(['Measure code'], 1)
df_2015['year'] = 2015
df_2015.head()

,recipient_name,EAFRD,EAGF,scheme,recipient_postcode,recipient_location,year
0,CYPROFRESH CITRUS SEDIGEP (P.O.) LTD,NaN,81818.45,III.5,4180,ΛΕΜΕΣΟΣ,2015
1,CYPROFRESH CITRUS SEDIGEP (P.O.) LTD,NaN,45271.93,III.5,4180,ΛΕΜΕΣΟΣ,2015
2,CYPROFRESH CITRUS SEDIGEP (P.O.) LTD,NaN,56893.72,III.5,4180,ΛΕΜΕΣΟΣ,2015
3,CYPROFRESH CITRUS SEDIGEP (P.O.) LTD,NaN,54618.98,III.5,4180,ΛΕΜΕΣΟΣ,2015
4,CYPROFRESH CITRUS SEDIGEP (P.O.) LTD,NaN,71666.05,ΙΙΙ.5,4186,ΥΨΩΝΑΣ,2015


In [41]:
df_2014 = pd.read_excel('BENEFICIARIES_2014_FINAL_v3.xlsx')
original_columns = list(df_2014.columns)
translations = ('CODE MEASURE (ABB CODE)',
               'CODE EPEXIGISIS MEASURE',
               'MEASURE', 'CODE APPLICANT', 'NAME / TRADE NAME', 'EAGF', 'EAFRD', 'POSTAL CODE', 'COMMUNITY / DISTRICT')
df_2014 = df_2014.rename(columns=dict(zip(original_columns, translations)))
df_2014 = df_2014.rename(columns={
    'COMMUNITY / DISTRICT': 'recipient_location',
    'POSTAL CODE': 'recipient_postcode',
    'NAME / TRADE NAME': 'recipient_name',
    'CODE EPEXIGISIS MEASURE': 'scheme'
}).drop(['CODE MEASURE (ABB CODE)', 'MEASURE'], 1)
df_2014['year'] = 2014
df_2014.head()

,scheme,CODE APPLICANT,recipient_name,EAGF,EAFRD,recipient_postcode,recipient_location,year
0,I.2,NaN,A & F PELOPIDA LTD,2000.00,NaN,8820,ΠΟΛΗ ΧΡΥΣΟΧΟΥΣ,2014
1,I.2,NaN,A & F PELOPIDA LTD,38984.33,NaN,8820,ΠΟΛΗ ΧΡΥΣΟΧΟΥΣ,2014
2,I.2,NaN,A & X (ΓΕΩΡΓΙΚΕΣ ΕΠΙΧΕΙΡΗΣΕΙΣ) ΛΙΜΙΤΕΔ,2000.00,NaN,7643,ΚΑΛΟ ΧΩΡΙΟ ΛΑΡΝΑΚΑΣ,2014
3,I.2,NaN,A & X (ΓΕΩΡΓΙΚΕΣ ΕΠΙΧΕΙΡΗΣΕΙΣ) ΛΙΜΙΤΕΔ,152.04,NaN,7643,ΚΑΛΟ ΧΩΡΙΟ ΛΑΡΝΑΚΑΣ,2014
4,I.2,NaN,A & X (ΓΕΩΡΓΙΚΕΣ ΕΠΙΧΕΙΡΗΣΕΙΣ) ΛΙΜΙΤΕΔ,37936.49,NaN,7643,ΚΑΛΟ ΧΩΡΙΟ ΛΑΡΝΑΚΑΣ,2014


In [51]:
df = pd.concat([df_2014, df_2015])
df.head()

,CODE APPLICANT,EAFRD,EAGF,recipient_location,recipient_name,recipient_postcode,scheme,year
0,NaN,NaN,2000.00,ΠΟΛΗ ΧΡΥΣΟΧΟΥΣ,A & F PELOPIDA LTD,8820,I.2,2014
1,NaN,NaN,38984.33,ΠΟΛΗ ΧΡΥΣΟΧΟΥΣ,A & F PELOPIDA LTD,8820,I.2,2014
2,NaN,NaN,2000.00,ΚΑΛΟ ΧΩΡΙΟ ΛΑΡΝΑΚΑΣ,A & X (ΓΕΩΡΓΙΚΕΣ ΕΠΙΧΕΙΡΗΣΕΙΣ) ΛΙΜΙΤΕΔ,7643,I.2,2014
3,NaN,NaN,152.04,ΚΑΛΟ ΧΩΡΙΟ ΛΑΡΝΑΚΑΣ,A & X (ΓΕΩΡΓΙΚΕΣ ΕΠΙΧΕΙΡΗΣΕΙΣ) ΛΙΜΙΤΕΔ,7643,I.2,2014
4,NaN,NaN,37936.49,ΚΑΛΟ ΧΩΡΙΟ ΛΑΡΝΑΚΑΣ,A & X (ΓΕΩΡΓΙΚΕΣ ΕΠΙΧΕΙΡΗΣΕΙΣ) ΛΙΜΙΤΕΔ,7643,I.2,2014


In [52]:
list(df.columns)

['CODE APPLICANT',
 'EAFRD',
 'EAGF',
 'recipient_location',
 'recipient_name',
 'recipient_postcode',
 'scheme',
 'year']

In [53]:
# There only ever is one value filled EAGF or EAFRD
((df[['EAGF', 'EAFRD']].fillna(0.0) == 0.0).sum(1) > 1).any()

False

In [54]:
add_to_df(df)

,recipient_location,recipient_name,recipient_postcode,scheme,year,country,currency,recipient_id,amount
0,ΠΟΛΗ ΧΡΥΣΟΧΟΥΣ,A & F PELOPIDA LTD,8820,I.2,2014,CY,EUR,CY-8820-A-F-PELOPIDA-LTD,2000.00
1,ΠΟΛΗ ΧΡΥΣΟΧΟΥΣ,A & F PELOPIDA LTD,8820,I.2,2014,CY,EUR,CY-8820-A-F-PELOPIDA-LTD,38984.33
2,ΚΑΛΟ ΧΩΡΙΟ ΛΑΡΝΑΚΑΣ,A & X (ΓΕΩΡΓΙΚΕΣ ΕΠΙΧΕΙΡΗΣΕΙΣ) ΛΙΜΙΤΕΔ,7643,I.2,2014,CY,EUR,CY-7643-A-X-GEORGIKES-EPIKHEIRESEIS-LIMITED,2000.00
3,ΚΑΛΟ ΧΩΡΙΟ ΛΑΡΝΑΚΑΣ,A & X (ΓΕΩΡΓΙΚΕΣ ΕΠΙΧΕΙΡΗΣΕΙΣ) ΛΙΜΙΤΕΔ,7643,I.2,2014,CY,EUR,CY-7643-A-X-GEORGIKES-EPIKHEIRESEIS-LIMITED,152.04
4,ΚΑΛΟ ΧΩΡΙΟ ΛΑΡΝΑΚΑΣ,A & X (ΓΕΩΡΓΙΚΕΣ ΕΠΙΧΕΙΡΗΣΕΙΣ) ΛΙΜΙΤΕΔ,7643,I.2,2014,CY,EUR,CY-7643-A-X-GEORGIKES-EPIKHEIRESEIS-LIMITED,37936.49


In [55]:
df.to_csv('cy_2014_2015.csv.gz', index=False, encoding='utf-8', compression='gzip')